In [1]:
from fastapi import FastAPI
from fastapi.testclient import TestClient
from langchain_core.runnables import ConfigurableFieldSpec, RunnablePassthrough
from langserve import add_routes
from typing import List

In [2]:
app = FastAPI()

chain = RunnablePassthrough()

add_routes(
    app,
    chain,
    path="/example",
)

# 結果が見づらいのでLangServeのロゴとパスのログを出力するイベントハンドラーを消す
app.router.on_startup.clear()

with TestClient(app) as client:
    req_body = {
        "input": "Hello, world!",
    }
    resp = client.post("http://testserver/example/invoke", json=req_body)

resp.json()

{'output': 'Hello, world!',
 'callback_events': [],
 'metadata': {'run_id': 'a8971b0c-537a-4824-b561-b76ed5b54d96'}}

## configurableを使う

In [3]:
from langchain_core.runnables.config import RunnableConfig


app = FastAPI()


class MyRunnablePassthrough(RunnablePassthrough):
    @property
    def config_specs(self) -> List[ConfigurableFieldSpec]:
        return [
            ConfigurableFieldSpec(id="foo", annotation=str),
            ConfigurableFieldSpec(id="bar", annotation=str),
        ]


chain = MyRunnablePassthrough() | (
    lambda input, config: {
        "input": input,
        "config": config["configurable"] if "configurable" in config else None,
    }
)

add_routes(
    app,
    chain,
    path="/example",
)

# 結果が見づらいのでLangServeのロゴとパスのログを出力するイベントハンドラーを消す
app.router.on_startup.clear()

with TestClient(app) as client:
    req_body = {
        "input": "Hello, world!",
        "config": {
            "configurable": {
                "foo": "FOO",
                "bar": "BAR",
                "baz": "BAZ",
                "qux": "QUX",
            }
        },
    }
    resp = client.post("http://testserver/example/invoke", json=req_body)

resp.json()

{'output': {'input': 'Hello, world!', 'config': {'bar': 'BAR', 'foo': 'FOO'}},
 'callback_events': [],
 'metadata': {'run_id': '3fa383d0-dabb-4e15-a5d4-c7310dc1a49c'}}